DSCI Final Project - Baseball Statistics

MLB Stats API: <br>
https://pypi.org/project/MLB-StatsAPI/ <br>
Wiki/Documentation: https://github.com/toddrob99/MLB-StatsAPI/wiki <br>
Usage: run 'pip install MLB-StatsAPI' <br><br>
Datasets:<br>
SFBB-Player-ID-Map.csv: from https://www.smartfantasybaseball.com/tools/ <br>
expected_stats.csv: from https://baseballsavant.mlb.com/leaderboard/expected_statistics?type=batter&year=2015&position=&team=&min=q <br>
Batting.csv: from https://www.kaggle.com/datasets/open-source-sports/baseball-databank?select=Batting.csv


In [159]:
# Setups
import pandas as pd
import plotly.express as px
import os
import statsapi as api
#import logging

In [160]:
data_location = "datasets/"
exp_stats_loc = "exp_stats/"

exp_stats = pd.read_csv(data_location + exp_stats_loc + "expected_stats_2015.csv")
batting_names = pd.read_csv(data_location + 'SFBB-Player-ID-Map.csv')
batting = pd.read_csv(data_location + 'Batting.csv')

In [161]:
#renames columns of exp_stats and makes it easier to view
all_exp_df = []
for year in range(2015, 2024):
    file_name = os.path.join(data_location, exp_stats_loc, 'expected_stats_' + str(year) + '.csv')
    data = pd.read_csv(file_name)
    all_exp_df.append(data)
exp_stats = pd.concat(all_exp_df, ignore_index=True)
exp_stats = exp_stats.rename(columns={'last_name':'Name',
                                      ' first_name':'to_drop',
                                      'player_id':'ID',
                                      'year':'Year',
                                      'pa':'Plate Appearances',
                                      'bip':'Balls In Play',
                                      'ba':'BA',
                                      'est_ba':'xBA',
                                      'est_ba_minus_ba_diff':'xBA - BA Diff',
                                      'slg':'Slg',
                                      'est_slg':'xSlg',
                                      'est_slg_minus_slg_diff':'xSlg - Slg Diff',
                                      'woba':'WOBA',
                                      'est_woba':'xWOBA',
                                      'est_woba_minus_woba_diff':'xWOBA - WOBA Diff'})
exp_stats['Name'] = (exp_stats['to_drop'] + ' ' + exp_stats['Name']).astype('str')
exp_stats = exp_stats.drop(columns =['to_drop'])
#exp_stats = exp_stats.set_index('Name')
exp_stats = exp_stats.replace(r"^ +| +$", r"", regex=True)

In [162]:
#renames columns of batting and makes it easier to view
batting = batting[batting.yearID >= 2005]
batting = batting.drop(columns=['stint', 'lgID'])
batting = batting.rename(columns = {'playerID':'Player ID',
                                    'yearID':'Year',
                                    'teamID':'Team',
                                    'G':'Games'})
#batting = batting.set_index('Player ID')

In [163]:
batting_final = pd.merge(batting_names[['IDPLAYER','PLAYERNAME', 'BATS']], batting,  right_on='Player ID', left_on='IDPLAYER')
batting_final = batting_final.rename(columns = {'PLAYERNAME':'Name'})
#batting_final = batting_final.set_index('Name')

In [164]:
display(batting_final)
display(exp_stats)

,IDPLAYER,Name,BATS,Player ID,Year,Team,Games,AB,R,H,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,aardsda01,David Aardsma,R,aardsda01,2006,CHN,45,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,aardsda01,David Aardsma,R,aardsda01,2007,CHA,25,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aardsda01,David Aardsma,R,aardsda01,2008,BOS,47,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,aardsda01,David Aardsma,R,aardsda01,2009,SEA,73,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aardsda01,David Aardsma,R,aardsda01,2010,SEA,53,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559,zobribe01,Ben Zobrist,B,zobribe01,2015,KCA,59,232.0,37.0,66.0,...,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0
9560,zuninmi01,Mike Zunino,R,zuninmi01,2013,SEA,52,173.0,22.0,37.0,...,14.0,1.0,0.0,16.0,49.0,0.0,3.0,0.0,1.0,5.0
9561,zuninmi01,Mike Zunino,R,zuninmi01,2014,SEA,131,438.0,51.0,87.0,...,60.0,0.0,3.0,17.0,158.0,1.0,17.0,0.0,4.0,12.0
9562,zuninmi01,Mike Zunino,R,zuninmi01,2015,SEA,112,350.0,28.0,61.0,...,28.0,0.0,1.0,21.0,132.0,0.0,5.0,8.0,2.0,6.0


,Name,ID,Year,Plate Appearances,Balls In Play,BA,xBA,xBA - BA Diff,Slg,xSlg,xSlg - Slg Diff,WOBA,xWOBA,xWOBA - WOBA Diff
0,Marcus Semien,543760,2022,724,547,0.248,0.243,0.005,0.429,0.394,0.035,0.317,0.306,0.011
1,Freddie Freeman,518692,2022,708,517,0.325,0.313,0.012,0.511,0.538,-0.027,0.393,0.403,-0.010
2,Trea Turner,607208,2022,708,527,0.298,0.276,0.022,0.466,0.432,0.034,0.350,0.335,0.015
3,Vladimir Guerrero Jr.,665489,2022,706,526,0.274,0.276,-0.002,0.480,0.462,0.018,0.351,0.348,0.003
4,Francisco Lindor,596019,2022,706,504,0.270,0.254,0.016,0.449,0.427,0.022,0.342,0.331,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,Zach McKinstry,656716,2023,64,44,0.271,0.286,-0.015,0.458,0.508,-0.050,0.336,0.360,-0.024
2259,Sam Hilliard,656541,2023,64,30,0.276,0.224,0.052,0.500,0.411,0.089,0.365,0.309,0.056
2260,David Hensley,682073,2023,63,33,0.143,0.170,-0.027,0.161,0.223,-0.062,0.197,0.230,-0.033
2261,Eduardo Escobar,500871,2023,63,46,0.172,0.223,-0.051,0.397,0.348,0.049,0.256,0.269,-0.013


In [165]:
#an unfortunately messy function that parses data returned by API calls, and converts it into a DataFrame
def parse_player_api_data(data):
    lines = data.split('\n')
    line = lines[0]

    name_start = 0
    name_end = line.find('"') - 1
    last_name_start = line.find(' ') + 1
    last_name_end = line.find(',', last_name_start)

    name = line[name_start:name_end+1]
    first_word_end = line.find(' ')
    if first_word_end == -1:
        name = line
    else:
        name = line[:first_word_end]

    last_name = line[last_name_start:last_name_end]
    nickname_start = last_name.find('"')
    nickname_end = last_name.find('"', nickname_start + 1)
    if nickname_start != -1 and nickname_end != -1:
        last_name = last_name[:nickname_start] + last_name[nickname_end+1:]\

    career_hitting = {}
    for line in lines[3:]:
        if line:
            key, value = line.split(': ')
            career_hitting[key] = value
    df = pd.DataFrame(career_hitting, index=[0])
    df['Name'] = name
    df['LastName'] = last_name
    df = df[['Name','LastName'] + list(career_hitting.keys())]
    
    numeric_cols = ['gamesPlayed', 'groundOuts', 'runs', 'doubles', 'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls',
                    'intentionalWalks', 'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
                    'groundIntoDoublePlay', 'numberOfPitches', 'plateAppearances', 'totalBases', 'rbi', 'leftOnBase',
                    'sacBunts', 'sacFlies']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)
    
    return df

    


In [166]:


df_for_api_call = exp_stats[exp_stats.Year == 2023].drop_duplicates(subset=['Name'], keep=False)
player_season = pd.DataFrame()
player_career = pd.DataFrame()

for playerID in df_for_api_call['ID']:
    #f.write(api.player_stats(playerID, group='hitting', type='season'))
    season_data = parse_player_api_data(api.player_stats(playerID, group='hitting', type='season'))
    career_data = parse_player_api_data(api.player_stats(playerID, group='hitting', type='career'))
    player_season = player_season.append(season_data)
    player_career = player_career.append(career_data)

display(player_season)
display(player_career)

C:\Users\HOMANJP20\AppData\Local\Temp\ipykernel_28784\3546598193.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_season = player_season.append(season_data)
C:\Users\HOMANJP20\AppData\Local\Temp\ipykernel_28784\3546598193.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_career = player_career.append(career_data)
C:\Users\HOMANJP20\AppData\Local\Temp\ipykernel_28784\3546598193.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_season = player_season.append(season_data)
C:\Users\HOMANJP20\AppData\Local\Temp\ipykernel_28784\3546598193.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_career = player_c

,Name,LastName,gamesPlayed,groundOuts,airOuts,runs,doubles,triples,homeRuns,strikeOuts,...,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun
0,Freddie,Freeman,32,29,35,25,9,0,5,25,...,145,63,14,50,0,2,.347,0.83,0,25.20
0,Alex,Verdugo,32,39,32,26,9,1,5,18,...,144,67,18,46,0,0,.336,1.22,0,26.00
0,Steven,Kwan,31,37,40,15,4,1,0,15,...,144,40,11,47,0,2,.306,0.93,0,-.--
0,Nico,Hoerner,30,42,36,23,5,1,2,14,...,142,53,17,34,0,1,.328,1.17,0,65.50
0,Trea,Turner,32,25,40,19,5,2,3,35,...,143,53,8,50,0,0,.330,0.63,0,45.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Zach,McKinstry,23,15,15,9,5,0,2,16,...,69,28,4,23,1,0,.341,1.00,0,31.00
0,Sam,Hilliard,22,7,8,10,4,0,3,28,...,65,29,6,21,0,0,.464,0.88,0,19.67
0,David,Hensley,19,16,11,9,1,0,0,23,...,66,9,2,35,0,0,.229,1.45,0,-.--
0,Eduardo,Escobar,18,19,17,8,2,1,3,14,...,63,23,9,45,1,1,.167,1.12,0,19.33


,Name,LastName,gamesPlayed,groundOuts,airOuts,runs,doubles,triples,homeRuns,strikeOuts,...,plateAppearances,totalBases,rbi,leftOnBase,sacBunts,sacFlies,babip,groundOutsToAirouts,catchersInterference,atBatsPerHomeRun
0,Freddie,Freeman,1756,1498,1676,1111,423,27,297,1439,...,7513,3310,1055,2442,0,50,.341,0.89,0,21.90
0,Alex,Verdugo,541,600,467,280,124,6,49,312,...,2101,834,219,789,1,18,.320,1.28,0,39.00
0,Steven,Kwan,178,186,232,104,29,8,6,75,...,782,265,63,264,2,6,.320,0.80,0,114.50
0,Nico,Hoerner,277,290,267,128,42,7,15,131,...,1037,367,118,337,0,7,.311,1.09,1,63.13
0,Trea,Turner,881,891,911,605,196,38,127,708,...,3880,1721,442,1185,2,17,.343,0.98,2,28.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Zach,McKinstry,144,98,94,50,21,3,14,121,...,433,155,47,170,4,2,.277,1.04,0,28.07
0,Sam,Hilliard,236,134,123,94,23,7,32,237,...,704,270,77,269,0,2,.290,1.09,0,19.66
0,David,Hensley,35,25,15,16,3,1,1,29,...,100,26,7,48,0,0,.298,1.67,0,87.00
0,Eduardo,Escobar,1282,993,1414,569,243,42,161,979,...,4875,1940,614,1996,14,41,.288,0.70,0,27.71
